# Optimizing Stardew Valley Purchases with Linear Programming

## Introduction

Stardew Valley is a managerial game in which the protagonist is the owner of a farm. Here, far far away from the chaothic city, the player can plant different seeds and harvest all the goods that is able to produce. The player can also manage a ranch or some other activities. <br>
While playing, at some point during the game play, in everyone's mind an eternal question popped:<br>
"*How can i become the richest farmer in Pellican Town and marry that hot chick, Leah?*"<br>

In this notebook we will see how to optimize all the purchases that the player will make while buying seeds, applying some sweet sweet **Linear Programming**.

![Leah](resources/leah.png)

## Modelization
In stardew valley, every plant can be bought in an emporium for a given cost. Every crop has a different price and different growing time. When a crop has completed its growing cycle, it can be harvested and sold, generating a revanue.<br>
In this context, different resources must be considered.
- *money* - Someone said somewhere:
    >The person who doesn't know where his next dollar is coming from usually doesn't know where his last dollar went.
    >
    Of course money is a resource. We have little money, and we want to make a lot of money. The game starts with a very very small amount, and it's our duty to harvest it and become richer and richer (and marry Leah).
    
- *space* - The space that we have at our disposal in the farm it's not infinite. Every crop will use one tile, and sometimes this space must be sacrified for other objects like scarerows or splinkers. Sadly, every penny in the world is not enough when no space can be used.

- *time* - I imagine that somewhere in the internet there's another good quote, but i'm too lazy. In Stardew Calley, time is essential, since every crop needs several days to complete its growth cycle. However, there is another limitation. The year is divided in 4 months of 28 days, one for each season. Every season has a set of allowed plants that can grow; if a green cabbage plant lasts till the end of the spring, it will be atrociously killed by the mighty spears of the summer Sun.


### Elements of the modelization

#### Constants
$P$ : the set of all the cultures. $p_{i}$ is the $i$-th crop<br>
$N$ : the number of available tiles <br>
$D$ : number of days available <br>
$M$ : total amount of money available <br>

$t_{i}$: time needed by the $i$-th culture to grow <br>
$c_{i}$: cost of the $i$-th culture <br>
$r_{i}$: revenue of the $i$-th culture <br>
$\beta_{i}$ : days needed for products to regrow. If the crop does not regraw, it's set to -1<br>

**if** $\beta_{i} \neq -1$<br> 
$\alpha_{i} = \lfloor\frac{D}{\beta_{i}}\rfloor$ : number of times that a crop will produce in the D days<br>
**else** $\alpha=1$

$max_{i}$: maximum number of seeds allowed for the culture <br>
$min_{i}$: minimum number of seeds allowed for the culture <br>
These two can be set to $+\infty$ and $0$, but can also be used in order to have some diversifiaction. <br>

#### Variables
$x_{i}$: amount of the $i$-th culture that needs to be purchased. 

#### Objective function
*maximize* $\pi = \sum_{i \in P}\alpha_{i} r_{i}x_{i} - \sum_{i \in P}c_{i}x_{i}$ 

#### Constaints
**tile availablity**; &nbsp; *the number of products cannot exceed the number of plantable tiles*<br>
$\sum_{i \in P}x_{i} \leq N$<br>

**time horizon constraint**; *If there is not enough time for this culture, it should not be planted*<br>
 **if** $t_{i} > D \rightarrow x_{i} = 0$

**total cost constraint**; *The total cost cannot exceed the total amount of money owned*
$\sum_{i \in P}c_{i}x_{i} \leq M$ 

 *the real problem is integer, but can be relaxed to linear if the amount of money is much larger*<br>
$\forall i \in P$ &nbsp; $x_{i} \geq 0, \in Natural Numbers$

## Code of the optimizer

The following class is ued to store all the information regarding a crop, such buy cost, sell cost, growth time and season in which the seed can be planted. The following lines of code retrieve these information from a JSON file.

In [1]:
class Crop:
    
    def __init__(self, name, buyCost, sellCost, growthTime):
        self.name = name
        self.buyCost = buyCost
        self.sellCost=sellCost
        self.growthTime = growthTime
        self.seasons = []
        self.regrow=-1
    
    def addSeason(self, season):
        self.seasons= self.seasons +(season)
        
    def setRegrowTime(self, regrow):
        self.regrow = regrow;
        
    def __str__(self):
        stringa ="Name={}\nBuy={}\nSell={}\ngrothTime={}".format(self.name, self.buyCost, self.sellCost, self.growthTime)
        if(self.regrow != -1):
            stringa= stringa +("\nregrow=" + str(self.regrow))
        stringa = stringa + "\nseasons:" + str(self.seasons)
        return stringa

In [2]:
import json 
from math import *
from pulp import *

In [3]:
jsonFile = open("resources/stardewConfig.json", "r")
data = json.load(jsonFile)

crops = data.get("crops") 

In [4]:
cropList = []
for crop in crops:
    x = Crop(crop.get("name"), crop.get("buy"), crop.get("sell"), sum([x for x in crop.get("stages")]))
    if(crop.get("regrow")):
        x.setRegrowTime(crop.get("regrow"))
    x.addSeason(crop.get("seasons"))    
    cropList.append(x)
    #print(x)
    #print("\n")

The followings are the input of the program.

$N$ : the number of available tiles <br>
$D$ : number of days available <br>
$M$ : total amount of money available <br>

In [5]:
D = 30
N = 120
M = 10000

**if** $\beta_{i} \neq -1$<br> 
$\alpha_{i} = \lfloor\frac{D}{\beta_{i}}\rfloor$ : number of times that a crop will produce in the D days<br>
**else** $\alpha=1$

In [6]:
alpha= {}
for crop in cropList:
    if crop.regrow == -1:
        alpha[crop.name] =1
    else:
        alpha[crop.name] = floor(D/crop.regrow)
    

Creation of the variables. The name of the variable is the name of the crop.

In [7]:
variables = {}
for crop in cropList:
    variables[crop.name]= LpVariable(crop.name, 0, cat="Integer")
    

In [8]:
prob = LpProblem("ProfitMaximization", LpMaximize)

*maximize* $\pi = \sum_{i \in P}\alpha_{i} r_{i}x_{i} - \sum_{i \in P}c_{i}x_{i}$ 

In [9]:
prob += lpSum([alpha[x.name]*x.sellCost*variables[x.name] for x in cropList]) -lpSum([x.buyCost*variables[x.name] for x in cropList])

**tile availablity**<br>
$\sum_{i \in P}x_{i} \leq N$<br>

In [10]:
prob += lpSum([variables[x.name] for x in cropList]) <= N

**time horizon constraint**<br>
 **if** $t_{i} > D \rightarrow x_{i} = 0$

In [11]:
for crop in cropList:
    if(crop.growthTime > D):
        prob += variables[crop.name] == 0

**total cost constraint**<br>
$\sum_{i \in P}c_{i}x_{i} \leq M$ 


In [12]:
prob += lpSum([x.buyCost*variables[x.name] for x in cropList]) <= M

In [13]:
status = prob.solve()

In [14]:
status

1

In [15]:
LpStatus[status]

'Optimal'

In [16]:
for v in prob.variables():
    if value(v)> 0:
        print(v.name,"=", value(v))

Ancient_Fruit = 4.0
Hops = 116.0


In [17]:
print("profit =",value(prob.objective))

profit = 86040.0


In [18]:
prob

ProfitMaximization:
MAXIMIZE
1500*Ancient_Fruit + 270*Blueberry + 195*Coffee_Bean + 200*Corn + 690*Hops + 360*Hot_Pepper + 170*Melon + 40*Poppy + 50*Radish + 160*Red_Cabbage + 350*Starfruit + 40*Summer_Spangle + -45*Sunflower + 370*Tomato + 15*Wheat + 0
SUBJECT TO
_C1: Ancient_Fruit + Blueberry + Coffee_Bean + Corn + Hops + Hot_Pepper
 + Melon + Poppy + Radish + Red_Cabbage + Starfruit + Summer_Spangle
 + Sunflower + Tomato + Wheat <= 120

_C2: 700 Ancient_Fruit + 80 Blueberry + 30 Coffee_Bean + 150 Corn + 60 Hops
 + 40 Hot_Pepper + 80 Melon + 100 Poppy + 40 Radish + 100 Red_Cabbage
 + 400 Starfruit + 50 Summer_Spangle + 125 Sunflower + 50 Tomato + 10 Wheat
 <= 10000

VARIABLES
0 <= Ancient_Fruit Integer
0 <= Blueberry Integer
0 <= Coffee_Bean Integer
0 <= Corn Integer
0 <= Hops Integer
0 <= Hot_Pepper Integer
0 <= Melon Integer
0 <= Poppy Integer
0 <= Radish Integer
0 <= Red_Cabbage Integer
0 <= Starfruit Integer
0 <= Summer_Spangle Integer
0 <= Sunflower Integer
0 <= Tomato Integer
0